<a href="https://colab.research.google.com/github/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [1]:
from run import *
from agent import AgentDQN, AgentUADQN
from StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf

# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [2]:
# Agent
args = Arguments(if_on_policy=False)
args.agent = AgentUADQN()

# Environment
tickers = 'TARA'  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1
initial_capital = 1e6
initial_stocks = 100
buy_cost_pct = 1e-3
sell_cost_pct = 1e-3
start_date = '2008-03-19'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False, if_save=True)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True, if_save=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = 2000
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = 2000
args.batch_size = 32
args.repeat_times = 4 # repeat_times * target_step == number of times we update before training
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = True
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)
args.target_step = 50 # number of exploration steps before training

# **Part 6: Train and Evaluate the Agent**

> The training and evaluating processes are all finished inside function **train_and_evaluate_mp()**, and the only parameter for it is **args**. It includes the fundamental objects in DRL:

*   agent,
*   environment.

> And it also includes the parameters for training-control:

*   batch_size,
*   target_step,
*   reward_scale,
*   gamma, etc.

> The parameters for evaluation-control:

*   break_step,
*   random_seed, etc.






In [3]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentUADQN/StockTradingEnv-v1-TARA_0
| Remove history
ID      Step      MaxR |    avgR      stdR       avgQ      avgL |  avgS  stdS
start exploring before training
Begin updating network before training
start training
explore environment
update environment: total_steps=100
0   5.00e+01      0.90 |
0   5.00e+01      0.90 |0.899288  0.000000   0.063605  2.271025 |  1258     0
explore environment
update environment: total_steps=150
0   1.00e+02      0.91 |
0   1.00e+02      0.91 |0.905226  0.000000   0.114587  3.925931 |  1258     0
explore environment
update environment: total_steps=200
0   1.50e+02      0.96 |
0   1.50e+02      0.96 |0.960562  0.000000   0.168561  4.304025 |  1258     0
explore environment
update environment: total_steps=250
0   2.00e+02      1.04 |
0   2.00e+02      1.04 |1.038102  0.000000   0.171238  4.450926 |  1258     0
explore environment
update environment: total_steps=300
0   2.50e+02      1.04 |1.036643  0.000000   0.205459  5.841272 |  125

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [4]:
args.env_eval.draw_cumulative_return(args, torch)

Loaded act: ./AgentUADQN/StockTradingEnv-v1-TARA_0
The initial captial is 1000000.0
The initial number of stocks is 100


[0.9983425233158169,
 0.9961392183875398,
 0.9934246501443925,
 0.9915414226198931,
 0.9915182530890498,
 0.9895614299578659,
 0.9873137156180467,
 0.9865868631349715,
 0.9842563338540927,
 0.9838682412536099,
 0.9828026999005823,
 0.9831815513894807,
 0.9833478364815604,
 0.9834092588647446,
 0.9843860228187283,
 0.9839027725701841,
 0.9815561785731193,
 0.982970396960659,
 0.9823683371206742,
 0.9821570861146618,
 0.9806418252615634,
 0.9807723990436966,
 0.9784088024901767,
 0.9767134568953275,
 0.9769110689769447,
 0.9769462983477727,
 0.9768058916820528,
 0.9774622378450031,
 0.9762689580078588,
 0.9767224549069735,
 0.976473640344648,
 0.9760650532594802,
 0.9735618668039577,
 0.9728241839464091,
 0.9732301535056573,
 0.9724375602423897,
 0.972975143682242,
 0.9720205299910051,
 0.9722037725465132,
 0.9722035657813756,
 0.9726578323391563,
 0.9732860692136533,
 0.9742262720730956,
 0.973497682384131,
 0.9739901550442646,
 0.9739496778393221,
 0.9750493085735927,
 0.97490089215547

In [5]:
args.env_eval.draw_cumulative_return_while_learning(args, torch)

Loaded act: ./AgentUADQN/StockTradingEnv-v1-TARA_0
The initial captial is 1000000.0
The initial number of stocks is 100
updating network: 49
updating network: 99
updating network: 149
updating network: 199
updating network: 249


/jupyteruser/31484685/STAT-234-project/agent.py:277: RuntimeWarning: invalid value encountered in sqrt
  aleatoric_uncertainties.append(np.sqrt(np.cov(net1[i].cpu().data.numpy(), net2[i].cpu().data.numpy())[0][1]))


updating network: 299
updating network: 349
updating network: 399
updating network: 449
updating network: 499
updating network: 549
updating network: 599
updating network: 649
updating network: 699
updating network: 749
updating network: 799
updating network: 849
updating network: 899
updating network: 949
updating network: 999
updating network: 1049
updating network: 1099
updating network: 1149
updating network: 1199
updating network: 1249


In [6]:
np.load('AgentDQN/StockTradingEnv-v1-TARA_0/cumulative_return.npy')[60:]

array([0.97643857, 0.97733649, 0.97687286, ..., 1.01973701, 1.01999423,
       1.01995129])

In [7]:
np.load('AgentDQN/StockTradingEnv-v1-TARA_0/recorder.npy')

array([[0.00000000e+00,           -inf, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [5.00000000e+01, 9.04169542e-01, 0.00000000e+00, 3.27200562e-01,
        3.76881519e-03],
       [1.00000000e+02, 9.39311769e-01, 1.11022302e-16, 3.98768038e-01,
        4.93733473e-02],
       [1.50000000e+02, 9.44646516e-01, 6.66133815e-16, 5.06502748e-01,
        4.16513048e-02],
       [2.00000000e+02, 1.00437479e+00, 2.22044605e-16, 5.56410551e-01,
        1.95107721e-02],
       [2.50000000e+02, 1.01995129e+00, 6.66133815e-16, 7.50232577e-01,
        3.02283801e-02],
       [3.00000000e+02, 1.01059015e+00, 0.00000000e+00, 6.93461776e-01,
        2.08338071e-02],
       [9.50000000e+02, 9.64251094e-01, 1.11022302e-16, 8.26295316e-01,
        1.71577297e-02],
       [1.60000000e+03, 9.34476498e-01, 1.11022302e-16, 7.88282752e-01,
        4.38189646e-03]])